# Script abaixo eu costumo utilizar para meus estudos, ele configura o spark no google collab e faz várias outras coisas para facilitar meus trabalhos

In [35]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark pandas numpy

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.2.2-bin-hadoop3.2')


from pyspark.sql import functions as func
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import warnings, re

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


# Instanciando Spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

--2023-10-06 18:07:03--  https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz.1’

spark-3.2.2-bin-had 100%[===================>] 287.16M  17.7MB/s    in 17s     

2023-10-06 18:07:21 (16.7 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz.1’ saved [301112604/301112604]



# Fazendo algumas configurações no ambiente.

In [46]:
project_id = 'future-graph-401118'

spark.conf.set('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar')
spark.conf.set('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery_2.12:0.23.0')
spark.conf.set('spark.hadoop.google.cloud.auth.service.account.enable', 'true')
spark.conf.set('spark.hadoop.google.cloud.auth.service.account.json.keyfile', '/content/your-service-account-key.json')

# Set the project ID
spark.conf.set('spark.executorEnv.BIGQUERY_PROJECT_ID', project_id)



## "auth" para nos autenticarmos, "bigquery" para instanciarmos o nosso client.


In [36]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated')

Authenticated


# Instaciando nosso client

In [37]:
client = bigquery.Client(project='future-graph-401118')

# Precisaremos usar o id do nosso projeto, o nome do dataset e qual tabela iremos salvar em uma dataframe pandas.

In [38]:
project_id = 'future-graph-401118'
dataset_id = 'canizela_nba'
table_id = 'draft_picks_from_duke_1'

# Criando a query e transformando-a em um dataframe pandas.

In [39]:
query = f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`"


df = client.query(query).to_dataframe()

# Salvando nosso df pandas em csv

In [40]:
df.to_csv('draft_picks_from_duke_1.csv', index=False)

# Lendo nosso csv em pyspark_df (eu sei que poderíamos ter lido diretamente do bigquery com pyspark, mas eu quis dar esse passo a mais)

In [41]:
spark_df = (spark.read.format("csv").options(header="true")
    .load("/content/draft_picks_from_duke_1.csv"))

In [42]:
spark_df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- _3P: string (nullable = true)
 |-- _3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG_: string (nullable = true)
 |-- _3P_: string (nullable = true)
 |-- FT_: string (nullable = true)
 |-- MP_G: string (nullable = true)
 |-- PTS_G: string (nullable = true)
 |-- TRB_G: string (nullable = true)
 |-- AS

# Salvando apenas algumas colunas do nosso dataframe no novo dataframe. Nosso objetivo aqui não é aprofundas no tratamento dos dados e sim mostrar que é possível usarmos python para trabalhar com as tabelas do Big Query.

In [25]:
new_spark_df = spark_df.select("Player", "Year", "Rd")

In [27]:
new_spark_df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)



# Criando o nome da tabela que vamos criar com nosso novo dataframe, enviando nosso df para o bigquery.

In [47]:
new_table_id = 'new_draft_picks_from_duke_1'

spark_df.write \
    .format('bigquery') \
    .option('table', f'{project_id}.{dataset_id}.{new_table_id}') \
    .save()

Py4JJavaError: ignored